### 필요한 라이브러리 불러오기 및 상수 선언

In [1]:
import pandas as pd
import urllib.request
from PIL import Image
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 앞뒤로 각각 몇 개의 데이터를 기준을 삼을 것인지

NUM_NEIGHBORS = 5       # 19년 이후 데이터 

### 데이터셋 가져오기

In [56]:
YOUTUBE_LIST = ['LabeledData_added','과학드림', '곽튜브', '리뷰엉이', '배말랭', '백종원', '보다', '비글부부', '서기채널', '셜록현준', '슈카월드', '알간지', '영국남자', '옐언니', '오킹TV', '오사카에사는사람들TV', '웅이', '육식맨', '주연', '지식해적단', '쯔양', '청담언니', '테스터훈', '트래블튜브', '피지컬갤러리', '햄지', '허팝', '혜진쓰', 'CreativeDen', 'ITSub잇섭', 'LeoJ Makeup', 'SOD', 'UNDERkg']

In [4]:
# path 설정
DATA_PATH = '/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_csv/'

### 함수 선언

In [5]:
def getLabels(df, n) :

    # input : dataset to get labels, number of neighbors to get mean of views
    # output : dataset with labels and mean of neighbors

    # create columns 'NeighborsMean' & 'label'
    df['NeighborsMean'] = 0
    df['label'] = 0
    df = df.astype({'NeighborsMean':'int'})

    for i in range(len(df)):
        mean = 0

        # [ : NUM_NEIGHBORS - 1 ]
        if i in range(0, n):
            mean = df.iloc[:i+n+1]['ViewCount'].mean()
        
        # [ NUM_NEIGHBORS : length - NUM_NEIGHBORS - 1 ]
        elif i in range(n, len(df)-n):
            mean = df.iloc[i-n:i+n+1]['ViewCount'].mean()
        
        # [ length - NUM_NEIGHBORS : ]
        else:
            mean = df.iloc[i-n:]['ViewCount'].mean()
        
        df['NeighborsMean'][i] = round(mean)
        if df['ViewCount'][i] >= mean:
            df['label'][i] = 1
    
    return df

In [6]:
#get image from url 
def url_to_img(dataset, IDX):
    for i in range(len(dataset)):
        img_id = dataset['Unnamed: 0'][i]
        img_url = dataset['HDThumbnailUrl'][i]

        if dataset['label'][i]==0:
            file_name = IMG_PATH + f"/0/{IDX+1}_{img_id}.png"
        else:
            file_name = IMG_PATH + f"/1/{IDX+1}_{img_id}.png"

        urllib.request.urlretrieve(img_url, file_name)
        print(f"{file_name}.png saved")   

### 라벨링 및 이미지 저장

In [7]:
fullData = pd.DataFrame({
    'ChannelName' : [],
    'VideoIndex': [],
    'VideoID' : [],
    'VideoTitle': [],
    'Label': [],
})

datas = []
zeros = []
ones = []

In [65]:
for idx in range(len(YOUTUBE_LIST)):
    NAME = YOUTUBE_LIST[idx]

    data = pd.read_csv(DATA_PATH + NAME + '.csv')

    # index 초기화 및 불필요한 column 제거
    data['Published'] = pd.to_datetime(data.Published)
    data = data.sort_values(by='Published')
    data.reset_index(inplace=True)
    data.drop(['index'], axis=1, inplace=True)

    dataLabeled = getLabels(data, NUM_NEIGHBORS)

    # 영상 정보 저장
    datas.append( len(dataLabeled) )
    zeros.append( len(dataLabeled[dataLabeled['label']==0]) )
    ones.append( len(dataLabeled[dataLabeled['label']==1]) )

    mainData = pd.DataFrame({
        'ChannelName' : [NAME for i in range(len(dataLabeled))],
        'VideoIndex': dataLabeled['Unnamed: 0'],
        'VideoID' : dataLabeled['VideoId'],
        'VideoTitle': dataLabeled['Title'],
        'Label':dataLabeled['label']
    })
    fullData = pd.concat([fullData, mainData])

    # 이미지 저장
    IMG_PATH = '/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/' + NAME

    os.makedirs(IMG_PATH, exist_ok=True)
    os.makedirs(IMG_PATH + '/0', exist_ok=True)
    os.makedirs(IMG_PATH + '/1', exist_ok=True)

    url_to_img(dataLabeled, idx)


/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/1/17_172.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/1/17_171.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/0/17_170.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/0/17_169.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/1/17_168.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/0/17_167.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/1/17_166.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/1/17_165.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/1/17_164.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/0/17_163.png.png saved
/Users/sea/YBIGTAlab/project0/git0/datasets/thumbnail_labeled/육식맨/0/17_162.png.png saved
/Users/sea/YBIGTAlab/

### 영상 정보 저장

len(fullData)

In [66]:
info = pd.DataFrame({
    'youtube' : YOUTUBE_LIST,
    'num_of_data' : datas,
    'label 0' : zeros,
    'label 1' : ones
})

In [67]:
info

,youtube,num_of_data,label 0,label 1
0,침착맨,1856,1245,611
1,과학드림,139,89,50
2,곽튜브,276,170,106
3,리뷰엉이,509,329,180
4,배말랭,316,175,141
5,백종원,496,320,176
6,보다,1154,787,367
7,비글부부,291,188,103
8,서기채널,180,117,63
9,셜록현준,112,71,41


In [84]:
info.to_csv('/Users/sea/YBIGTAlab/project0/data_information.csv')
fullData.to_csv('/Users/sea/YBIGTAlab/project0/full_data_label.csv')